In [1]:
import os 
import time
import requests  
import json  
import openai
import numpy as np
from openai import OpenAI

api_key = os.environ["AI_ATTACK_API_KEY"]  

%reload_ext autoreload
%autoreload 2

# define useful functions

In [2]:
# Define the API URL and headers

def call_ai_attack_gpt( prompt ):
    headers = {
        "Authorization": api_key,  # Replace with your API key
        "Content-Type": "application/json",
    }
    
    # Define the request payload (adjust as needed for your use case)
    data = {
        "messages": [
            {'role': 'user', 'content': prompt },
            {'role': 'system', 'content': "You are a senior software engineer who is experienced in software requirement classification! "}      
        ],
        'temperature': 0,  
    
    }

    url = "https://openai-aiattack-msa-001333-swedencentral-01-freeexperiment.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview"
    # Make the POST request
    response = requests.post(url, headers=headers, json=data)
    
    # # Check if the request was successful
    # if response.status_code == 200:
    #     print("Request was successful!")
    #     print(response.json())
    # else:
    #     print(f"Request failed with status code {response.status_code}")
    #     print("Response:", response.text)

    return response.json()["choices"][0]["message"]["content"]


In [3]:
url = "https://openai-aiattack-msa-001333-westus3-01-freeexperimental.openai.azure.com/openai/deployments/gpt-4o-2024-05-13/chat/completions?api-version=2024-10-01-preview"
prompt = ("Please classify the given requirement into 'functional requirement' or 'non-functional requirement'. "
                "The answer should be in format {the given requirement: functional requirement or non-functional requirement}."
                "The given requirement: The system shall refresh the display every 60 seconds." )
call_ai_attack_gpt( prompt)

'{The system shall refresh the display every 60 seconds: non-functional requirement}'

In [4]:
def get_completion(prompt_list, save_path):
    """load the prompt_list, get the completion from llm, and save the results."""

    start_time = time.time()  
    
    completion_list = []
    for i, prompt in enumerate(prompt_list):
        #print("The prompt: " + prompt + "\n######")
     
        j = 0
        while j < 10:
            try:   
                completion = call_ai_attack_gpt( prompt )
                #print(completion)
                completion_list.append(completion)
                if i % 50 == 0:
                    print(i, completion)
                break
        
            except: #InvalidRequestError: in case that the prompt and output reach the token limitation
                #print("{}th prompt, Connection error: try again automatically".format(i) )
                time.sleep(2)  # stop 1 second to avoid error
            j = j+1   
            
        if j == 50:
            print("{}th prompt. At least one prompt did not get the result.".format(j))
            break

        # completion = get_response_siemens_api(prompt)
        # #print(completion)
        # completion_list.append(completion)
        # break

    save_completion_list(save_path, completion_list)

    # calculate the execution time
    end_time = time.time()  
    print("execution time: {}".format(end_time - start_time), "second")

In [5]:
def read_prompt_list(path):
    """read the saved list of prompts 
    input
        path, string path to save the list
    """
    with open(path, 'r') as file:  
        content = file.read()  
    prompt_list_read = content.split('\n\n\n')   

    return prompt_list_read


In [6]:
def save_completion_list(path, completion_list):
    """save the constructed prompts with few-shot examples in a list 
    input
        path, string path to save the list
        prompt_list, list of constructed prompts from the first requirement/sentence in test dataset to the last one
    """
    with open(path, 'w', newline='\n') as file:  
        for i, completion in enumerate(completion_list):
            if i+1 == len(completion_list):
                file.write(completion)
            else:     
                file.write(completion + "\n\n\n")

# get completion on promise dataset

## binary classification

In [11]:
import os  
  
current_path = os.getcwd()  
current_path

'/Users/yongjiantang/Desktop/tang/code/re_genai/llm_siemens_api_gpt_mistral'

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_0_bi.txt"
get_completion(prompt_list, save_path)

0 Functional


In [9]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_5_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Non-functional
execution time: 131.39737391471863 second


In [10]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_10_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Functional
execution time: 131.64453792572021 second


In [7]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_20_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
50 Non-functional
100 Functional
execution time: 136.29599785804749 second


In [12]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_40_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Non-functional
execution time: 131.07602286338806 second


In [13]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_80_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Non-functional
execution time: 133.9329218864441 second


In [8]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_120_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_120_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
50 Non-functional
100 Functional
execution time: 162.7880358695984 second


In [14]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_160_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Non-functional
execution time: 158.38647890090942 second


#### binary embedding

In [15]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_0_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Functional
execution time: 128.7198760509491 second


In [16]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_5_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Functional
execution time: 132.9164400100708 second


In [17]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_10_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Functional
execution time: 130.2189428806305 second


In [18]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_20_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Non-functional
execution time: 133.7374861240387 second


In [19]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_40_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Non-functional
execution time: 139.584645986557 second


In [20]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_80_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Non-functional
execution time: 160.78017807006836 second


In [13]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_120_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_120_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
50 Non-functional
100 Functional
execution time: 176.2581090927124 second


In [21]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_160_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Non-functional
execution time: 174.14355421066284 second


#### binary tfidf

In [22]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_0_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Functional
execution time: 130.27217197418213 second


In [23]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_5_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Non-functional
execution time: 131.31780791282654 second


In [24]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_10_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Non-functional
execution time: 135.04482889175415 second


In [25]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_20_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Functional
execution time: 131.51315689086914 second


In [7]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_40_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Non-functional
120 Non-functional
execution time: 138.01073098182678 second


In [8]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_80_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Non-functional
execution time: 147.18111181259155 second


In [15]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_120_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_120_bi.txt"
get_completion(prompt_list, save_path)


0 Functional
50 Non-functional
100 Functional
execution time: 168.90804505348206 second


In [9]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_160_bi.txt"
get_completion(prompt_list, save_path)

0 Functional
20 Non-functional
40 Functional
60 Functional
80 Non-functional
100 Functional
120 Non-functional
execution time: 175.54652404785156 second


# PURE binary 

#### PURE binary random

In [10]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_random_0_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Functional
300 Functional
350 Functional
400 Functional
450 Functional
500 Non-functional
550 Non-functional
600 Non-functional
execution time: 735.8549046516418 second


In [11]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_random_5_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Functional
350 Functional
400 Functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 643.9163591861725 second


In [12]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_random_10_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Functional
350 Functional
400 Functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 669.6705040931702 second


In [16]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_random_20_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Functional
350 Functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 658.8623781204224 second


In [17]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_random_40_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Functional
350 Functional
400 Functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 656.7644679546356 second


In [18]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_random_80_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Functional
350 Non-functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 669.541079044342 second


In [7]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_120_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_random_120_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Functional
350 Non-functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 708.6756472587585 second


In [8]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_random_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_random_160_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Functional
350 Non-functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 787.6393117904663 second


#### PURE binary embedding

In [9]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_embedding_0_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Functional
300 Functional
350 Functional
400 Functional
450 Functional
500 Non-functional
550 Non-functional
600 Non-functional
execution time: 669.3030622005463 second


In [10]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_embedding_5_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Non-functional
350 Functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 673.3067870140076 second


In [11]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_embedding_10_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Non-functional
350 Functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 691.7403950691223 second


In [7]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_embedding_20_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Functional
300 Non-functional
350 Functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 688.3288769721985 second


In [8]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_embedding_40_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Non-functional
350 Functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 704.3295860290527 second


In [9]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_embedding_80_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Non-functional
350 Functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 720.0857992172241 second


In [10]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_120_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_embedding_120_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Non-functional
350 Functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 822.3165621757507 second


In [11]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_embedding_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_embedding_160_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Non-functional
350 Functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 885.6270339488983 second


#### PURE binary tfidf

In [7]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_0_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_tfidf_0_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Functional
300 Functional
350 Functional
400 Functional
450 Functional
500 Non-functional
550 Non-functional
600 Non-functional
execution time: 631.6310977935791 second


In [8]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_5_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_tfidf_5_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Non-functional
350 Functional
400 Non-functional
450 Functional
500 Non-functional
550 Non-functional
600 Non-functional
execution time: 629.4923741817474 second


In [9]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_10_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_tfidf_10_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Non-functional
350 Functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 651.9405047893524 second


In [10]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_20_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_tfidf_20_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Functional
300 Non-functional
350 Non-functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 667.4180910587311 second


In [7]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_40_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_tfidf_40_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Functional
300 Non-functional
350 Non-functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 671.9923558235168 second


In [8]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_80_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_tfidf_80_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Non-functional
350 Non-functional
400 Non-functional
450 Functional
500 Non-functional
550 Non-functional
600 Non-functional
execution time: 688.0920460224152 second


In [7]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_120_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_tfidf_120_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Non-functional
350 Non-functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 742.7052609920502 second


In [8]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_pure_tfidf_160_bi.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_pure_tfidf_160_bi.txt"
get_completion(prompt_list, save_path)

0 Non-functional
50 Non-functional
100 Functional
150 Functional
200 Non-functional
250 Non-functional
300 Non-functional
350 Functional
400 Non-functional
450 Functional
500 Non-functional
550 Functional
600 Non-functional
execution time: 822.4338319301605 second


# multi-class classification # PROMISE_NEW dataset multi-class eva

#### promise multi-class random

In [7]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_0_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_random_0.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 141.60802006721497 second


In [8]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_5_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_random_5.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 135.91384482383728 second


In [9]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_10_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_random_10.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 140.91921496391296 second


In [10]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_20_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_random_20.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 140.12513828277588 second


In [11]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_40_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_random_40.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 138.27656316757202 second


In [12]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_80_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_random_80.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 145.5010359287262 second


In [13]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_120_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_random_120.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 171.65326714515686 second


In [14]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_random_160_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_random_160.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 174.6509268283844 second


#### promise multiclass embedding

In [15]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_0_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_embedding_0.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 134.73550629615784 second


In [16]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_5_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_embedding_5.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 139.83812522888184 second


In [17]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_10_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_embedding_10.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 137.19032287597656 second


In [18]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_20_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_embedding_20.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 142.79167222976685 second


In [19]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_40_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_embedding_40.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 145.3158619403839 second


In [20]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_80_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_embedding_80.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Functional Suitability
100 Functional Suitability
execution time: 165.45671105384827 second


In [21]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_120_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_embedding_120.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Functional Suitability
100 Functional Suitability
execution time: 179.33378314971924 second


In [22]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_embedding_160_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_embedding_160.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 184.65365934371948 second


#### promise multiclass tf-idf

In [23]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_0_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_tfidf_0.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 136.47507786750793 second


In [24]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_5_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_tfidf_5.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 136.59660005569458 second


In [25]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_10_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_tfidf_10.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Usability
100 Functional Suitability
execution time: 141.55880999565125 second


In [26]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_20_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_tfidf_20.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Functional Suitability
100 Functional Suitability
execution time: 140.97017312049866 second


In [27]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_40_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_tfidf_40.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Functional Suitability
100 Functional Suitability
execution time: 145.93206095695496 second


In [28]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_80_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_tfidf_80.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Functional Suitability
100 Functional Suitability
execution time: 152.34541630744934 second


In [29]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_120_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_tfidf_120.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Functional Suitability
100 Functional Suitability
execution time: 178.50329279899597 second


In [30]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_new_tfidf_160_mul.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_new_tfidf_160.txt"
get_completion(prompt_list, save_path)

0 Functional Suitability
50 Functional Suitability
100 Functional Suitability
execution time: 180.60286283493042 second


# PROMISE original dataset multi-class eva

### random

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_0.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_5.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_10.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_20.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_40.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_80.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_120.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_120.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_random_160.txt"
get_completion(prompt_list, save_path)

### embedding

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_0.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_5.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_10.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_20.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_40.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_80.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_120.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_120.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_embedding_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_embedding_160.txt"
get_completion(prompt_list, save_path)

### tfidf

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_0.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_5.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_10.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_20.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_40.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_80.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_120.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_120.txt"
get_completion(prompt_list, save_path)

In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_tfidf_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_promise_tfidf_160.txt"
get_completion(prompt_list, save_path)

# get completion on nfr dataset

## multi-class classification

#### nfr multi_class random

In [7]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_random_0.txt"
get_completion(prompt_list, save_path)

0 Performance
20 Security
40 Portability
60 Security
80 Performance
100 Availability
120 Scalability
140 Security
160 Performance
180 Security
200 Performance
220 Security
240 Security
260 Security
280 Performance
300 Maintainability
320 Performance
340 Portability
360 Security
380 Portability
400 Performance
420 Fault Tolerance
440 Security
460 Maintainability
480 Performance
500 Fault Tolerance
520 The given requirement does not fall under any of the specified nonfunctional software requirement categories. It is a functional requirement related to data manipulation in Python.
540 Performance
560 Performance
580 Performance
600 Performance
620 The given requirement does not fit into any of the specified nonfunctional software requirement categories (Availability, Fault Tolerance, Maintainability, Performance, Portability, Scalability, Security). It appears to be a question about the functionality or usage of a specific tool (VisualVM sampler) rather than a nonfunctional requirement.
6

In [97]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_random_5.txt"
get_completion(prompt_list, save_path)

execution time: 3478.666871070862 second


In [114]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_random_10.txt"
get_completion(prompt_list, save_path)

execution time: 3468.873160123825 second


In [115]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_random_20.txt"
get_completion(prompt_list, save_path)

execution time: 3486.1135108470917 second


In [116]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_random_40.txt"
get_completion(prompt_list, save_path)

execution time: 3484.3390600681305 second


In [117]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_random_80.txt"
get_completion(prompt_list, save_path)

execution time: 3487.0319340229034 second


In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_120.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_random_120.txt"
get_completion(prompt_list, save_path)

In [118]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_random_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_random_160.txt"
get_completion(prompt_list, save_path)

execution time: 27610.913210868835 second


#### nfr multi_class embedding

In [119]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_embedding_0.txt"
get_completion(prompt_list, save_path)

execution time: 9217.068017959595 second


In [120]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_embedding_5.txt"
get_completion(prompt_list, save_path)

execution time: 3708.531285047531 second


In [121]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_embedding_10.txt"
get_completion(prompt_list, save_path)

execution time: 3597.099881887436 second


In [122]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_embedding_20.txt"
get_completion(prompt_list, save_path)

execution time: 3647.655138015747 second


In [123]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_embedding_40.txt"
get_completion(prompt_list, save_path)

execution time: 3484.010708808899 second


In [8]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_embedding_80.txt"
get_completion(prompt_list, save_path)

execution time: 3463.9130859375 second


In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_120.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_embedding_120.txt"
get_completion(prompt_list, save_path)

In [9]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_embedding_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_embedding_160.txt"
get_completion(prompt_list, save_path)

execution time: 3492.20218706131 second


#### nfr multi_class tfidf

In [10]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_0.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_tfidf_0.txt"
get_completion(prompt_list, save_path)

execution time: 3478.533807039261 second


In [11]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_5.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_tfidf_5.txt"
get_completion(prompt_list, save_path)

execution time: 3480.6994071006775 second


In [12]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_10.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_tfidf_10.txt"
get_completion(prompt_list, save_path)

execution time: 35353.59642314911 second


In [13]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_20.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_tfidf_20.txt"
get_completion(prompt_list, save_path)

execution time: 3509.2056109905243 second


In [14]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_40.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_tfidf_40.txt"
get_completion(prompt_list, save_path)

execution time: 3484.5739438533783 second


In [15]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_80.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_tfidf_80.txt"
get_completion(prompt_list, save_path)

50th prompt. At least one prompt did not get the result.
execution time: 2838.492636203766 second


In [ ]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_120.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_tfidf_120.txt"
get_completion(prompt_list, save_path)

In [16]:
path_in = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_nfr_tfidf_160.txt"
prompt_list = read_prompt_list(path_in)
save_path = "/Users/yongjiantang/Desktop/tang/code/re_genai/completions/gpt_4o/completion_nfr_tfidf_160.txt"
get_completion(prompt_list, save_path)

50th prompt. At least one prompt did not get the result.
execution time: 251.89404487609863 second
